In [36]:
# Attempt to rollup most of NSE programs into ONE

# STATUS: Incomplete (Trial with index options)
# Run-time: Unknown

# Dependencies:
#  - List of indexes and equities
#  - underlying's ohlc history for standard deviation and Black Scholes

#***          Start ib_insync (run once)       *****
#_______________________________________________

from ib_insync import *
util.startLoop()
ib = IB().connect('127.0.0.1', 7496, clientId=5) # kavi tws live
# ib = IB().connect('127.0.0.1', 4001, clientId=5) # kavi IBG live
# ib = IB().connect('127.0.0.1', 7499, clientId=5) # kavi tws paper-trade
# ib = IB().connect('127.0.0.1', 4004, clientId=5) # kavi IBG paper-trade

In [ ]:
%%time
#***      Imports    *****
import pandas as pd
import numpy as np
from itertools import product
import utils

#***    Paths and Variables    ***
datapath = r'./zdata/'
market = 'NSE'
deletesym = ['USDINR']   #ibSymbols to be deleted

#***    Get scrips from IBKR   ***
df_ib = pd.DataFrame([]) # Initialize

# from IBKR's symbols page
url = "https://www.interactivebrokers.com.hk/en/index.php?"
url = url+"f=2222&exch=nse&showcategories=OPTGRP&page="

for i in range(4):
    df_ib = df_ib.append(df_ib.append(pd.read_html(url+str(i), header=0)[2]))

# Exit program if dataframe is empty
if df_ib.empty:
    print('Interactive Brokers dataframe is empty!!!')
    sys.exit(0)
    
df_ib = df_ib.drop_duplicates().reset_index(drop=True)
all_symbols = list(df_ib['IB Symbol'])

# Qualify Stock contracts
stocks = [Stock(symbol=s, exchange=market) for s in all_symbols]
stock_contracts = ib.qualifyContracts(*stocks)
stock_symbols = [s.symbol for s in stock_contracts]

# Qualify Index contracts
idx_symbols = [a for a in all_symbols if a not in stock_symbols]

indexes = [Index(symbol=s, exchange='NSE') for s in idx_symbols]
index_contracts = ib.qualifyContracts(*indexes)
index_symbols = [i.symbol for i in index_contracts]

In [11]:
# All qualified symbols and contracts
symbols = stock_symbols + index_symbols
contracts = stock_contracts + index_contracts

In [2]:
#****     DATA LIMITER HERE   ****
#_________________________________
symbols = stock_symbols[0:2] + [index_symbols[0]] # DATA LIMITER!!
contracts = stock_contracts[0:2] + [index_contracts[0]]

Started to throttle requests
Error 200, reqId 2651: No security definition has been found for the request, contract: Stock(symbol='BANKNIFTY', exchange='NSE')
Stopped to throttle requests
Error 200, reqId 2765: No security definition has been found for the request, contract: Stock(symbol='NIFTY50', exchange='NSE')
Error 200, reqId 2826: No security definition has been found for the request, contract: Stock(symbol='USDINR', exchange='NSE')
Unknown contract: Stock(symbol='BANKNIFTY', exchange='NSE')
Unknown contract: Stock(symbol='NIFTY50', exchange='NSE')
Unknown contract: Stock(symbol='USDINR', exchange='NSE')


Wall time: 31.4 s


In [ ]:
# import importlib
# importlib.reload(utils)
blk = 25
[utils.getprice(contract=c, ib=ib) 
 for i in range(0, len(contracts), blk) 
 for c in contracts[i:i+blk]]

In [33]:
for i in range(0, len(contracts), blk):
    for c in contracts[i:i+blk]:
        print(utils.getprice(contract=c, ib=ib))
        ib.sleep(0.1)

Error 162, reqId 2630: Historical Market Data Service error message:Historical data request pacing violation, contract: Stock(conId=44652144, symbol='ACC', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='ACC', tradingClass='ACC')


IndexError: list index out of range

In [47]:
ib.reqHistoricalData(
            contracts[0],
            endDateTime='',
            durationStr='60 S',
            barSizeSetting='1 secs',
            whatToShow='ASK',
            useRTH=False,
            formatDate=1,
            keepUpToDate=True)

Error 162, reqId 2640: Historical Market Data Service error message:Historical data request pacing violation, contract: Stock(conId=44652144, symbol='ACC', exchange='NSE', primaryExchange='NSE', currency='INR', localSymbol='ACC', tradingClass='ACC')


[]

In [53]:
help(ib.flexreport)

AttributeError: 'IB' object has no attribute 'flexreport'

In [ ]:
df_q.contract.apply(get_price)

In [ ]:
#***   Error catching for list comprehension ***
import numpy as np
def catch(func, handle=lambda e : e, *args, **kwargs):
    '''List comprehension error catcher'''
    try:
        return func(*args, **kwargs)
    except Exception as e:
        np.nan
df_q['closeprice'] = [catch(lambda: getprice(c)) for c in df_q.contract]

In [ ]:
get_price(df_q.contract[3])

In [ ]:
df_q['closeprice'] = [get_price(c) for c in df_q.contract]

In [ ]:
# ***** Get list with Mlots + Margin% from 5paisa
paisaurl = "https://www.5paisa.com/5pit/spma.asp"
df_paisa = pd.read_html(paisaurl, header=0)[1].drop_duplicates(subset='Symbol')

df_paisa1 = df_paisa[['Symbol', 'Mlot', 'TotMgn%']].sort_values('Symbol').reset_index(drop=True)

# Rename Symbol and Margin fields
df_paisa1 = df_paisa1.rename(columns={'Symbol': 'nseSymbol', 'TotMgn%': 'marginpct'})

# Convert columns to numeric and make margin to pct
df_paisa1 = df_paisa1.apply(pd.to_numeric, errors='ignore')
df_paisa1.marginpct = df_paisa1.marginpct.div(100)

In [ ]:
#***    make option chains and qualify   ***
chains = {s: ib.reqSecDefOptParams(s.symbol, '', s.secType, s.conId) for s in symbols}
chains

df_und = util.df(chains) # dataframe of underlying chains

tup_chains = [([k.symbol], tuple(i.expirations), tuple(i.strikes), ['P']) 
 for k, v in chains.items() 
 for i in v]

# create options without USDINR scrip
options = [Option(p[0], p[1], p[2], p[3], 'NSE') 
           for tup in tup_chains
           for p in product(*tup)]

# qualify the options
nse_qo = [t for i in range(0, len(options), 25) for t in ib.qualifyContracts(*options[i: i+25])]

df_qo = util.df(nse_qo) # dataframe for qualified options

In [ ]:
contract = nse_qo[0]
order = Order(action='SELL', orderType='MKT', totalQuantity=20)
ib.whatIfOrder(contract, order)

In [ ]:
ib.reqSecDefOptParams(futFopExchange='NSE', underlyingConId=337009725, underlyingSecType='IND', underlyingSymbol = 'BANKNIFTY')

In [ ]:
ib.disconnect()

# To-do
* something is wrong in the qual_options. Fix it.
* integration with standard deviation
* remove unnecessary options (Put)
* integration with black scholes
* integration with lot and margin
* generate orders
* place orders (what-if) to check margins (duplicate)
* check latest prices
* screen most valueable RoM and PoP
* place orders